In [1]:
# a few tricks for doing sub-parts of this process
# remove silent sounds from a list
# samples = [sample for sample in samples if np.abs(sample).max() > 0]
# normalize peaks to 1 sounds
# samples = (samples.T / np.abs(samples).max(axis=1)).T

In [2]:
import numpy as np
from utils import list_all_files, ffmpeg_load_audio
from multiprocessing import Pool

In [3]:
def load_sample(fn, sr=None,
                max_length=None, fixed_length=None, normalize=True):
    if fn == '': # ignore empty filenames
        return None
    audio, _ = ffmpeg_load_audio(fn, sr, mono=True)
    duration = len(audio)
    if duration == 0: # ignore zero-length samples
        return None
    if max_length and duration >= max_length: # ignore long samples
        return None
    if fixed_length:
        audio.resize(fixed_length)
    max_val = np.abs(audio).max()
    if max_val == 0: # ignore completely silent sounds
        return None
    if normalize:
        audio /= max_val
    return (fn, audio, duration)

In [4]:
# input_dir = '/Users/kyle/Music/Samples'
input_dir = '/Volumes/Big Data/sound/freesound'
sr = 44100

In [7]:
files = list(list_all_files(input_dir, ['.mp3']))
len(files)

121877

In [8]:
def load_sample_closure(fn):
    return load_sample(fn, sr=sr, fixed_length=1*sr)
#     return load_sample(fn, sr=sr, max_length=3*sr, fixed_length=1*sr)

In [ ]:
p = Pool(4) # must be immediately before p.map
%time results = p.map(load_sample_closure, files)

In [7]:
valid = filter(None, results)
filenames = [x[0] for x in valid]
samples = [x[1] for x in valid]
durations = [x[2] for x in valid]
%time samples = np.array(samples)

In [8]:
print len(valid)

28488


In [29]:
output_dir = 'input/28k'
np.savetxt(output_dir + '/filenames.txt', filenames, fmt='%s')
np.savetxt(output_dir + '/durations.txt', durations, fmt='%i')
%time np.save(output_dir + '/samples.npy', samples)

CPU times: user 333 ms, sys: 2min 13s, total: 2min 13s
Wall time: 2min 20s
